In [2]:
import tkinter as tk
import xlwings as xw
from tkinter import filedialog, simpledialog, messagebox
import tkinter.ttk as ttk
import openpyxl
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
import os

class App:
    def __init__(self, master):
        self.master = master
        self.master.title('Gerador de Orçamento')

        self.excel_path = ''
        self.word_path = ''

        self.entries = {}
        self.items = []
        self.selected_item_index = None  # Para rastrear o item selecionado para edição

        campos = [
            'Cliente', 'Responsável', 'Cidade', 'Contato', 'Email', 'Telefone', 'Proposta',
            'Horas trabalhadas', 'Qtd Técnicos 1',
            'KM Ida/Volta', 'Qtd Veículos 1',
            'Diárias Hotel', 'Qtd Técnicos 2',
            'Refeições', 'Qtd Técnicos 3',
            'Pedágios', 'Qtd Veículos 2',
            'Motor', 'Modelo Motor', 'N° Série Motor',
            'Gerador', 'Modelo Gerador', 'N° Série Gerador',
            'Tensão', 'USCA', 'Potência', 'TAG'
        ]

        row = 0
        for campo in campos:
            label = tk.Label(master, text=campo)
            label.grid(row=row, column=0, sticky='w')
            entry = tk.Entry(master)
            entry.grid(row=row, column=1)
            self.entries[campo] = entry
            row += 1

        self.item_tree = ttk.Treeview(master, columns=('Código', 'Descrição', 'Unidade', 'Quantidade', 'Compra', 'Margem'), show='headings')
        # Ajustando a largura das colunas (aumentando a da Descrição)
        self.item_tree.column('Código', width=80)
        self.item_tree.column('Descrição', width=300)  # Aumentei bastante a largura
        self.item_tree.column('Unidade', width=80, anchor='center') # Centralizei o texto
        self.item_tree.column('Quantidade', width=100, anchor='center') # Centralizei o texto
        self.item_tree.column('Compra', width=120)
        self.item_tree.column('Margem', width=100)

        for col in ('Código', 'Descrição', 'Unidade', 'Quantidade', 'Compra', 'Margem'):
            self.item_tree.heading(col, text=col)

        # Posicionando a Treeview no centro e expandindo o espaço de digitação
        self.item_tree.grid(row=0, column=2, rowspan=row, padx=10, pady=10, sticky='nsew')

        # Configurando o peso da coluna para que ela se expanda com a janela
        master.grid_columnconfigure(2, weight=1)
        master.grid_rowconfigure(0, weight=1) # Permite que a Treeview expanda verticalmente também

        btn_frame = tk.Frame(master)
        btn_frame.grid(row=row, column=0, columnspan=3)

        tk.Button(btn_frame, text='Selecionar Excel', command=self.select_excel).pack(side='left')
        tk.Button(btn_frame, text='Selecionar Word', command=self.select_word).pack(side='left')
        self.add_item_btn = tk.Button(btn_frame, text='Adicionar Item', command=self.open_item_dialog)
        self.add_item_btn.pack(side='left')
        self.edit_item_btn = tk.Button(btn_frame, text='Editar Item', command=self.open_item_dialog, state='disabled')
        self.edit_item_btn.pack(side='left')
        tk.Button(btn_frame, text='Remover Item', command=self.remove_item).pack(side='left')
        tk.Button(btn_frame, text='Gerar Arquivos', command=self.generate_files).pack(side='left')

    def select_excel(self):
        path = filedialog.askopenfilename(filetypes=[("Arquivos Excel", "*")])
        if path:
            self.excel_path = path

    def select_word(self):
        path = filedialog.askopenfilename(filetypes=[("Arquivos Word", "*.docx")])
        if path:
            self.word_path = path

    def on_item_select(self, event):
        selected_item = self.item_tree.selection()
        if selected_item:
            self.edit_item_btn.config(state='normal')
            self.selected_item_index = self.item_tree.index(selected_item[0])
        else:
            self.edit_item_btn.config(state='disabled')
            self.selected_item_index = None

    def open_item_dialog(self):
        dialog = tk.Toplevel(self.master)
        dialog.title('Adicionar/Editar Item')

        # Aumentando o tamanho da janela de diálogo (ajuste conforme necessário)
        dialog.geometry('400x250')

        labels = ['Código', 'Descrição', 'Unidade', 'Quantidade', 'Compra', 'Margem']
        entries = {}
        row = 0
        for label_text in labels:
            label = tk.Label(dialog, text=label_text)
            label.grid(row=row, column=0, sticky='w', padx=3, pady=3)
            entry = tk.Entry(dialog)
            entry.grid(row=row, column=1, padx=1, pady=1, sticky='ew') # 'ew' faz o campo expandir horizontalmente
            entries[label_text] = entry
            # Se for o campo de descrição, vamos dar mais largura
            if label_text == 'Descrição':
                entries[label_text].config(width=50) # Aumenta a largura do campo de descrição
            row += 1

        if self.selected_item_index is not None:
            selected_item_data = self.items[self.selected_item_index]
            for i, label_text in enumerate(labels):
                entries[label_text].insert(0, selected_item_data[i])

        def save_item():
            new_item = [entries[label].get() for label in labels]
            if self.selected_item_index is not None:
                self.items[self.selected_item_index] = new_item
                self.selected_item_index = None
            else:
                self.items.append(new_item)
            self.refresh_items()
            dialog.destroy()
            self.edit_item_btn.config(state='disabled')

        save_button = tk.Button(dialog, text='Salvar', command=save_item)
        save_button.grid(row=row, column=0, columnspan=2, padx=5, pady=10)

        cancel_button = tk.Button(dialog, text='Cancelar', command=dialog.destroy)
        cancel_button.grid(row=row + 1, column=0, columnspan=2, padx=5, pady=5)

        # Centralizando a janela de diálogo
        dialog.update_idletasks()
        width = dialog.winfo_width()
        height = dialog.winfo_height()
        screen_width = self.master.winfo_screenwidth()
        screen_height = self.master.winfo_screenheight()
        x = (screen_width - width) // 2
        y = (screen_height - height) // 2
        dialog.geometry(f'+{x}+{y}')

        dialog.transient(self.master)
        dialog.grab_set()
        self.master.wait_window(dialog)

    def remove_item(self):
        selected = self.item_tree.selection()
        if not selected:
            messagebox.showerror('Erro', 'Nenhum item selecionado.')
            return

        idx = self.item_tree.index(selected[0])
        del self.items[idx]
        self.refresh_items()
        self.edit_item_btn.config(state='disabled')

    def refresh_items(self):
        for i in self.item_tree.get_children():
            self.item_tree.delete(i)
        for item in self.items:
            self.item_tree.insert('', 'end', values=item)

    def generate_files(self):
        if not self.excel_path or not self.word_path:
            messagebox.showerror('Erro', 'Selecione os arquivos antes.')
            return

        app = xw.App(visible=False)
        wb_xw = None
        try:
            wb_xw = app.books.open(self.excel_path)
            ws1_xw = wb_xw.sheets['CALCULOS SERVIÇOS E DESPESAS']
            ws2_xw = wb_xw.sheets['LISTA DE MATERIAIS']

            # Preenchendo planilha 1 com xlwings (seu código original)
            planilha1_mapping_xw = {
                'Cliente': 'A3', 'Cidade': 'B3', 'Contato': 'D3', 'Email': 'E3', 'Telefone': 'F3',
                'Horas trabalhadas': 'A7', 'Qtd Técnicos 1': 'B7',
                'KM Ida/Volta': 'A11', 'Qtd Veículos 1': 'B11',
                'Diárias Hotel': 'A13', 'Qtd Técnicos 2': 'B13',
                'Refeições': 'A15', 'Qtd Técnicos 3': 'B15',
                'Pedágios': 'A17', 'Qtd Veículos 2': 'B17'
            }
            for field, cell in planilha1_mapping_xw.items():
                value = self.entries[field].get()
                if value:
                    ws1_xw.range(cell).value = value

            # Preenchendo planilha 2 com xlwings (seu código original)
            planilha2_mapping_xw = {
                'Motor': 'B2', 'Modelo Motor': 'B3', 'N° Série Motor': 'B4',
                'Gerador': 'E2', 'Modelo Gerador': 'E3', 'N° Série Gerador': 'E4',
                'Tensão': 'G2', 'USCA': 'G3', 'Potência': 'G4', 'TAG': 'J2'
            }
            for field, cell in planilha2_mapping_xw.items():
                value = self.entries[field].get()
                if value:
                    ws2_xw.range(cell).value = value

            # *** ESCREVENDO OS ITENS NA PLANILHA 2 ***
            start_row = 7
            for i, item in enumerate(self.items):
                row_num = start_row + i
                # Assumindo a ordem das colunas: Código, Descrição, Unidade, Quantidade, Compra, Margem
                ws2_xw.range(f'B{row_num}').value = item[0]  # Código
                ws2_xw.range(f'D{row_num}').value = item[1]  # Descrição
                ws2_xw.range(f'H{row_num}').value = item[2]  # Unidade
                ws2_xw.range(f'I{row_num}').value = item[3]  # Quantidade
                ws2_xw.range(f'M{row_num}').value = item[4]  # Compra
                ws2_xw.range(f'P{row_num}').value = item[5]  # Margem

            # Agora preenchendo o Word
            # Agora preenchendo o Word
            doc = Document(self.word_path)

            replace_dict_header = {
                '{{PROPOSTA}}': self.entries['Proposta'].get(),
                # Adicione outros placeholders de cabeçalho aqui se precisar
            }

            for section in doc.sections:
                header = section.header
                for paragraph in header.paragraphs:
                    for key, val in replace_dict_header.items():
                        if key in paragraph.text:
                            new_text = paragraph.text.replace(key, str(val))
                            paragraph.clear()
                            paragraph.add_run(new_text)

            placeholder_paragraph = None
            for paragraph in doc.paragraphs:
                if '{{materiais}}' in paragraph.text:
                    placeholder_paragraph = paragraph
                    break

            # *** LENDO OS DADOS DA TABELA DO EXCEL (INTERVALO ESPECÍFICO) ***
            table_range = 'D28:K42'
            all_table_data = ws2_xw.range(table_range).value
            # Remove linhas completamente vazias
            table_data = [row for row in all_table_data if any(cell is not None for cell in row)]

            if table_data and placeholder_paragraph:
                # Títulos da tabela
                headers = ['Descrição', 'Unidade', 'Quantidade', 'Preço unitário', 'Preço total']

                # Criar a tabela no Word
                table = doc.add_table(rows=1, cols=len(headers))
                table.style = 'Table Grid'

                # Adicionar os títulos na primeira linha
                hdr_cells = table.rows[0].cells
                for i, header in enumerate(headers):
                    hdr_cells[i].text = header
                    hdr_cells[i].paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
                    hdr_cells[i].paragraphs[0].style.font.bold = True

                if table.columns:
                    from docx.shared import Inches
                    table.columns[0].width = Inches(4) # Defina a largura desejada em polegadas

                # Adicionar os dados nas linhas subsequentes
                for row_data in table_data:
                    row_cells = table.add_row().cells
                    data_indices = [0, 4, 5, 6, 7] # Mapeamento correto dos dados

                    for i, header_index in enumerate(data_indices):
                        cell_value = row_data[header_index]
                        cell_text = str(cell_value if cell_value is not None else '')

                        # Formatar como monetário as colunas de preço unitário e total
                        if headers[i] in ['Preço unitário', 'Preço total'] and cell_value is not None:
                            try:
                                cell_text = f'R${float(cell_value):.2f}'
                            except (ValueError, TypeError):
                                pass # Mantém o valor original se não for um número

                        row_cells[i].text = cell_text
                        row_cells[i].paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER # Centraliza o conteúdo da célula

                # *** INSERIR A TABELA ANTES DO PARÁGRAFO DO PLACEHOLDER ***
                placeholder_paragraph._p.addnext(table._tbl)

                # *** ADICIONAR UM PARÁGRAFO EM BRANCO DEPOIS DA TABELA (AGORA INSERIDA) ***
                doc.add_paragraph()

                # Remover o parágrafo placeholder
                placeholder_paragraph.clear()

            elif placeholder_paragraph:
                placeholder_paragraph.clear()

            total_materiais_excel = ws1_xw.range('J19').value
            total_atendimento_excel = ws1_xw.range('J21').value
            total_geral_excel = ws1_xw.range('J23').value

            replace_dict = {k: v for k, v in {
                '{{CLIENTE}}': self.entries['Cliente'].get(),
                '{{responsavel}}': self.entries['Responsável'].get(),
                '{{e-mail}}': self.entries['Email'].get(),
                '{{CONTATO}}': self.entries['Contato'].get(),
                '{{PROPOSTA}}': self.entries['Proposta'].get(),
                '{{total_materiais}}': total_materiais_excel,
                '{{total_atendimento}}': total_atendimento_excel,
                '{{total}}': total_geral_excel
            }.items() if k != '{{materiais}}'}

            for p in doc.paragraphs:
                for key, val in replace_dict.items():
                    if key in p.text:
                        if key in ['{{total_materiais}}', '{{total_atendimento}}', '{{total}}']:
                            new_text = p.text.replace(key, f'R${float(val):.2f}')
                            p.clear()
                            run = p.add_run(new_text)
                            run.font.size = Pt(12)  # Aumenta a fonte para 12 pontos (ajuste conforme necessário)
                            run.font.bold = True   # Coloca o texto em negrito
                        else:
                            new_text = p.text.replace(key, str(val))
                            p.clear()
                            p.add_run(new_text)

            save_dir = filedialog.asksaveasfilename(
                defaultextension=".xlsx",
                filetypes=[("Arquivos Excel", "*.xlsx"), ("Todos os arquivos", "*.*")],
                title="Salvar orçamento Excel como..."
            )
            if save_dir:
                wb_xw.save(save_dir)

            save_doc_path = filedialog.asksaveasfilename(
                defaultextension=".docx",
                filetypes=[("Documentos Word", "*.docx"), ("Todos os arquivos", "*.*")],
                title="Salvar orçamento Word como..."
            )
            if save_doc_path:
                doc.save(save_doc_path)
                messagebox.showinfo('Sucesso', 'Arquivos gerados e salvos com sucesso!')
            else:
                messagebox.showinfo('Atenção', 'Operação de salvar Word cancelada.')
                if save_dir:
                    messagebox.showinfo('Sucesso', 'Arquivo Excel salvo com sucesso!')
                return

        except Exception as e:
            messagebox.showerror('Erro', f'Ocorreu um erro ao processar os arquivos: {e}')
        finally:
            if wb_xw:
                wb_xw.close()
            if app:
                app.quit()

if __name__ == '__main__':
    root = tk.Tk()
    app = App(root)
    root.mainloop()


NameError: name 'psutil' is not defined

In [95]:
pip install xlwings

Note: you may need to restart the kernel to use updated packages.


In [10]:
import xlwings as xw
print(xw.__path__)


['C:\\Users\\Geramaster\\anaconda3\\Lib\\site-packages\\xlwings']
